In [3]:
from utils import *

BASE_PATH = '../data/world'
NODES_PATH = BASE_PATH + '/world_nodes.txt'
EDGES_PATH = BASE_PATH + '/world_edges.txt'

GraphBuilder = GraphBuilder(NODES_PATH, EDGES_PATH)
graph = GraphBuilder.build_graph()

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

def cluster_and_add_warehouses(graph, warehouse_type, n_clusters):
    # Extract node coordinates for clustering
    node_coordinates = np.array([(node.x, node.y) for node in graph.nodes])

    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(node_coordinates)
    
    # Initialize clusters
    clusters = [[] for _ in range(n_clusters)]

    # Assign nodes to clusters
    for i, label in enumerate(kmeans.labels_):
        clusters[label].append(graph.nodes[i])

    # Calculate centroids
    centroids = [(sum(node.x for node in cluster) / len(cluster), 
                  sum(node.y for node in cluster) / len(cluster)) for cluster in clusters]

    # Create WarehouseNodes and add them to the graph
    for i, centroid in enumerate(centroids):
        new_warehouse = WarehouseNode(f'Warehouse_{i}', centroid[0], centroid[1], warehouse_type)
        graph.add_node(new_warehouse)

    # Connect WarehouseNodes with original clusters
    for i, centroid in enumerate(centroids):
        cluster_nodes = clusters[i]
        for node in cluster_nodes:
            weight = ((node.x - centroid[0])**2 + (node.y - centroid[1])**2)**0.5
            edge_to_warehouse = Edge(node, graph.get_node_by_id(f'Warehouse_{i}'), weight)
            graph.add_edge(edge_to_warehouse)

    return graph


In [ ]:
def assign_warehouses(graph):
        local_criteria, regional_criteria, central_criteria = 100, 10, 3

        graph = cluster_and_add_warehouses(graph, 'local', local_criteria)
        graph = cluster_and_add_warehouses(graph, 'regional', regional_criteria)
        graph = cluster_and_add_warehouses(graph, 'central', central_criteria)

        graph.plot_graph()
        return graph

In [ ]:
graph = assign_warehouses(graph)